In [ ]:
!pip install xgboost
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.model_selection import KFold
from sklearn.metrics import matthews_corrcoef,accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, f1_score, precision_score
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="pandas")
warnings.simplefilter(action='ignore', category=Warning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
!pip install gdown
import gdown
# File ID from the Google Drive link
file_id = '1LgDTq4lF5FskiWkyAu7a4hMYSuhFCV-l'
destination = 'mushroom_train.csv'  # Replace with your desired file name and extension
gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=False)

# File ID from the Google Drive link
file_id = '1dlWqwY_0Pg3h8eZyQ80kkH6htpQn5W6o'
destination = 'mushroom_test.csv'  # Replace with your desired file name and extension
gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=False)

# File ID from the Google Drive link
file_id = '1-E9vUgsplqT3rc2_yFIQZE9O4cMi-Fo0'
destination = 'mushroom_sample_submission.csv'  # Replace with your desired file name and extension
gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=False)

# File ID from the Google Drive link
file_id = '18oSB_b0ub_htk9XWHbSaWmYVrZLU6NQJ'
destination = 'new_data.csv'  # this is secondary mushroom dataset combined rows with compitition dataset
gdown.download(f'https://drive.google.com/uc?id={file_id}', destination, quiet=False)



In [ ]:
# Load the datasets
train_df = pd.read_csv('mushroom_train.csv')
df_test = pd.read_csv('mushroom_test.csv')
df_sample_submission = pd.read_csv('mushroom_sample_submission.csv')
df= pd.read_csv("new_data.csv")
ids = df_test['id']
train_df = train_df.drop(columns=['id'], axis=1)
df_test = df_test.drop(columns=['id'], axis=1)

In [ ]:
df = df.drop_duplicates()

In [ ]:
import pandas as pd

# Assuming df_train is your training DataFrame and `class` is the target feature
# Assuming that each categorical feature is already converted to 'category' dtype

# List of categorical columns (excluding the target column 'class')
categorical_columns = df.select_dtypes(include=['object']).columns.difference(['class'])

# Function to calculate the percentage of each class label ('e', 'p') for each label in the feature
def calculate_class_percentage(df, feature, target='class'):
    percentages = df.groupby([feature, target]).size().unstack().fillna(0)
    percentages = percentages.div(percentages.sum(axis=1), axis=0) * 100
    return percentages

# Iterate through each categorical column and calculate the percentages
for feature in categorical_columns:
    print(f"Class percentages for feature '{feature}':")
    percentages = calculate_class_percentage(df, feature)
    print(percentages)
    print()

In [ ]:
# Initialize dictionaries to store the results for each feature
unknown_e = {}
unknown_p = {}

# Features to print full rows for
features_to_print = ['cap-shape', 'cap-surface', 'cap-color',
    'does-bruise-or-bleed', 'gill-attachment',
    'gill-spacing', 'gill-color', 'stem-root',
    'stem-surface', 'stem-color', 'veil-type',
    'veil-color', 'has-ring', 'ring-type',
    'spore-print-color', 'habitat', 'season']

# Function to calculate the percentage of each class label ('e', 'p') for each label in the feature
def calculate_class_percentage(df, feature, target='class'):
    percentages = df.groupby([feature, target]).size().unstack().fillna(0)
    percentages = percentages.div(percentages.sum(axis=1), axis=0) * 100
    return percentages

# Iterate through each categorical column and calculate the percentages
for feature in features_to_print:
    if feature in features_to_print:
        percentages = calculate_class_percentage(df, feature)

        # Extract labels where the percentage is 100 for class 'e'
        unknown_e[feature] = percentages[percentages['e'] == 100].index.tolist()

        # Extract labels where the percentage is 100 for class 'p'
        unknown_p[feature] = percentages[percentages['p'] == 100].index.tolist()

# Create and assign dynamic variables for each feature
for feature in features_to_print:
    # Define variable names dynamically
    var_name_e = f"{feature}_unknown_e"
    var_name_p = f"{feature}_unknown_p"

    # Assign the lists to dynamically created variable names
    globals()[var_name_e] = unknown_e.get(feature, [])
    globals()[var_name_p] = unknown_p.get(feature, [])

# Print the dynamically created variables to verify
for feature in features_to_print:
    var_name_e = f"{feature}_unknown_e"
    var_name_p = f"{feature}_unknown_p"
    print(f"{var_name_e}: {globals().get(var_name_e)}")
    print(f"{var_name_p}: {globals().get(var_name_p)}")
    print()

# Create replacement mappings for each feature
replacement_mappings = {}
for feature in features_to_print:
    var_name_e = f"{feature}_unknown_e"
    var_name_p = f"{feature}_unknown_p"

    # Create replacement mappings only if there are unknown labels
    if globals().get(var_name_e):
        replacement_mapping_e = {label: 'unknown-e' for label in globals().get(var_name_e)}
        replacement_mappings[feature] = replacement_mapping_e
        df[feature] = df[feature].replace(replacement_mapping_e)
        df_test[feature] = df_test[feature].replace(replacement_mapping_e)

    if globals().get(var_name_p):
        replacement_mapping_p = {label: 'unknown-p' for label in globals().get(var_name_p)}
        replacement_mappings.setdefault(feature, {}).update(replacement_mapping_p)
        df[feature] = df[feature].replace(replacement_mapping_p)
        df_test[feature] = df_test[feature].replace(replacement_mapping_p)

# If you want to verify the final replacement mappings
print(replacement_mappings)

In [ ]:
# List of columns to encode
columns_to_encode = [
    'cap-shape', 'cap-surface', 'cap-color', 'gill-spacing', 'gill-attachment',
    'does-bruise-or-bleed', 'gill-color', 'season', 'habitat', 'spore-print-color',
    'ring-type', 'has-ring', 'veil-color', 'veil-type', 'stem-surface', 'stem-root','stem-color'
]

# Convert all values to strings and fill NaN with '-1'
df[columns_to_encode] = df[columns_to_encode].astype(str).fillna('-1')
df_test[columns_to_encode] = df_test[columns_to_encode].astype(str).fillna('-1')

# Initialize LabelEncoder
le = LabelEncoder()

# Encode the features for both df and test_data
for col in columns_to_encode:
    combined_data = pd.concat([df[col], df_test[col]], axis=0)  # Combine df and test_data
    le.fit(combined_data)  # Fit on the combined data
    df[col] = le.transform(df[col])
    df_test[col] = le.transform(df_test[col])

In [ ]:
df.fillna(-1, inplace=True)
df_test.fillna(-1, inplace=True)

In [ ]:
df['class'] = df['class'].replace('e',0)
df['class'] = df['class'].replace('p',1)

In [ ]:
label = 'class'
X = df.drop(columns=[label], axis=1)
y = df[label]

X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [ ]:
### LGBM
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef
import pandas as pd
import numpy as np

# Best parameters for LightGBM
best_params = {
    'num_leaves': 227, 'learning_rate': 0.06737922153052582, 'n_estimators': 357, 'subsample_for_bin': 213892, 'min_child_samples': 292, 'reg_alpha': 4.5070762908503614e-07, 'reg_lambda': 0.00017079711660762664, 'colsample_bytree': 0.45636430645506504, 'subsample': 0.6819598716671345, 'max_depth': 14,'device':'GPU', 'verbosity':-1,
    'n_jobs': -1,
    'gpu_platform_id': 0,  # Adjust if necessary
    'gpu_device_id': 0  # Adjust if necessary
}

# Initialize LightGBM with the best parameters
best_model2 = lgb.LGBMClassifier(**best_params)

# Initialize arrays to hold OOF predictions
oof_preds_lgbm = np.zeros(len(X))

# Perform 4-fold stratified cross-validation for OOF predictions
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

for train_ind, valid_ind in skf.split(X, y):
    X_train, X_val = X.iloc[train_ind], X.iloc[valid_ind]
    y_train, y_val = y.iloc[train_ind], y.iloc[valid_ind]
    
    # Train the model
    best_model2.fit(X_train, y_train)
    
    # Predict probabilities and generate OOF predictions
    oof_preds_lgbm[valid_ind] = best_model2.predict_proba(X_val)[:, 1]

# Convert OOF probabilities to binary class labels
oof_preds_binary = (oof_preds_lgbm > 0.5).astype(int)

# Calculate the MCC score
mcc = matthews_corrcoef(y, oof_preds_binary)
print(f"Matthews Correlation Coefficient (OOF): {mcc}")

# Re-train the model with best parameters on the entire training data
best_model2.fit(X, y)

# Print feature importances
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_model2.feature_importances_
}).sort_values(by='Importance', ascending=False)
print("\nFeature importances for LightGBM:")
print(importance_df.head(20))

# Predict on test data
test_pred_prob = best_model2.predict_proba(df_test)[:, 1]

# Convert probabilities to binary class labels
test_pred_binary = (test_pred_prob > 0.5).astype(int)

# Prepare submission
df_sample_submission['predictions'] = test_pred_binary
df_sample_submission['predictions'] = df_sample_submission['predictions'].replace({0: 'e', 1: 'p'})
df_sample_submission.head()

# Save the final DataFrame to CSV
output_file = 'final_predictions_lgbm.csv'
df_sample_submission.to_csv(output_file, index=False)

# Load the CSV file to modify
df3 = pd.read_csv(output_file)

# Drop the 'class' column if it exists
if 'class' in df3.columns:
    df3.drop(columns=['class'], inplace=True)

# Save the updated DataFrame to a new CSV file
output_file = 'final_predictions_no_class_new12_lgbm.csv'
df3.to_csv(output_file, index=False)

print(f"'class' column dropped. Updated file saved as '{output_file}'")
#Matthews Correlation Coefficient (OOF): 0.9848833003490562

Matthews Correlation Coefficient (OOF): 0.9848833003490562  , cv=10

Submission and Description

Public Score

Select

final_predictions_no_class_new12_lgbm (1).csv
Complete · 30s ago
0.98487



In [ ]:
### LGBM
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef
import pandas as pd
import numpy as np

# Best parameters for LightGBM
best_params = {
  'num_leaves': 929, 'learning_rate': 0.010704218899215131, 'n_estimators': 2492, 'subsample_for_bin': 263922, 'min_child_samples': 58, 'reg_alpha': 0.11039306751778567, 'reg_lambda': 4.8106602020701585e-09, 'colsample_bytree': 0.34381132307476014, 'subsample': 0.8996454730099417, 'max_depth': 14,'device':'GPU',
    'n_jobs': -1,
    'gpu_platform_id': 0,  # Adjust if necessary
    'gpu_device_id': 0  # Adjust if necessary 
}
    
    

# Initialize LightGBM with the best parameters
best_model2 = lgb.LGBMClassifier(**best_params)

# Initialize arrays to hold OOF predictions
oof_preds_lgbm = np.zeros(len(X))

# Perform 4-fold stratified cross-validation for OOF predictions
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

for train_ind, valid_ind in skf.split(X, y):
    X_train, X_val = X.iloc[train_ind], X.iloc[valid_ind]
    y_train, y_val = y.iloc[train_ind], y.iloc[valid_ind]
    
    # Train the model
    best_model2.fit(X_train, y_train)
    
    # Predict probabilities and generate OOF predictions
    oof_preds_lgbm[valid_ind] = best_model2.predict_proba(X_val)[:, 1]

# Convert OOF probabilities to binary class labels
oof_preds_binary = (oof_preds_lgbm > 0.5).astype(int)

# Calculate the MCC score
mcc = matthews_corrcoef(y, oof_preds_binary)
print(f"Matthews Correlation Coefficient (OOF): {mcc}")

# Re-train the model with best parameters on the entire training data
best_model2.fit(X, y)

# Print feature importances
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_model2.feature_importances_
}).sort_values(by='Importance', ascending=False)
print("\nFeature importances for LightGBM:")
print(importance_df.head(20))

# Predict on test data
test_pred_prob = best_model2.predict_proba(df_test)[:, 1]

# Convert probabilities to binary class labels
test_pred_binary = (test_pred_prob > 0.5).astype(int)

# Prepare submission
df_sample_submission['predictions'] = test_pred_binary
df_sample_submission['predictions'] = df_sample_submission['predictions'].replace({0: 'e', 1: 'p'})
df_sample_submission.head()

# Save the final DataFrame to CSV
output_file = 'final_predictions_lgbm.csv'
df_sample_submission.to_csv(output_file, index=False)

# Load the CSV file to modify
df3 = pd.read_csv(output_file)

# Drop the 'class' column if it exists
if 'class' in df3.columns:
    df3.drop(columns=['class'], inplace=True)

# Save the updated DataFrame to a new CSV file
output_file = 'final_predictions_no_class_new12_lgbm.csv'
df3.to_csv(output_file, index=False)

print(f"'class' column dropped. Updated file saved as '{output_file}'")
#Matthews Correlation Coefficient (OOF): 0.9848833003490562

above lgbm with 10 cv and with best params i got lb score as 98503

best_params = {
  'num_leaves': 929, 'learning_rate': 0.010704218899215131, 'n_estimators': 2492, 'subsample_for_bin': 263922, 'min_child_samples': 58, 'reg_alpha': 0.11039306751778567, 'reg_lambda': 4.8106602020701585e-09, 'colsample_bytree': 0.34381132307476014, 'subsample': 0.8996454730099417, 'max_depth': 14,'device':'GPU',
    'n_jobs': -1,
    'gpu_platform_id': 0,  # Adjust if necessary
    'gpu_device_id': 0  # Adjust if necessary 
}

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import matthews_corrcoef
from xgboost import XGBClassifier
import pandas as pd

# Best parameters found from Optuna (from previous runs)
best_params = {
   
    'enable_categorical': True,
    'tree_method': 'hist',
    'device': 'cuda',
    'n_estimators': 360,         
    'learning_rate': 0.1,           
    'max_depth': 17,                
    'colsample_bytree': 0.4,         
    'min_child_weight': 2,           
    'reg_lambda': 67,                
    'subsample': 0.98,              
    'num_parallel_tree': 4,
}

# Initialize XGBoost with the best parameters
best_model1 = XGBClassifier(**best_params)

# Initialize arrays to hold OOF predictions
oof_preds_xgboost = np.zeros(len(X))

# Perform 4-fold stratified cross-validation for OOF predictions
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=9)

for train_ind, valid_ind in skf.split(X, y):
    X_train, X_val = X.iloc[train_ind], X.iloc[valid_ind]
    y_train, y_val = y.iloc[train_ind], y.iloc[valid_ind]
    
    # Train the model
    best_model1.fit(X_train, y_train)
    
    # Predict probabilities and generate OOF predictions
    oof_preds_xgboost[valid_ind] = best_model1.predict_proba(X_val)[:, 1]

# Convert OOF probabilities to binary class labels
oof_preds_binary = (oof_preds_xgboost > 0.5).astype(int)

# Calculate the MCC score
mcc = matthews_corrcoef(y, oof_preds_binary)
print(f"Matthews Correlation Coefficient (OOF): {mcc}")

# Re-train the model with best parameters on the entire training data
best_model1.fit(X, y)

# Print feature importances
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_model1.feature_importances_
}).sort_values(by='Importance', ascending=False)
print("\nFeature importances for XGBoost:")
print(importance_df.head(20))

# Predict on test data
test_pred_prob = best_model1.predict_proba(df_test)[:, 1]

# Convert probabilities to binary class labels
test_pred_binary = (test_pred_prob > 0.5).astype(int)

# Prepare submission
df_sample_submission['predictions'] = test_pred_binary
df_sample_submission['predictions'] = df_sample_submission['predictions'].replace({0: 'e', 1: 'p'})
df_sample_submission.head()

# Save the final DataFrame to CSV
output_file = 'final_predictions_xgboost.csv'
df_sample_submission.to_csv(output_file, index=False)

# Load the CSV file to modify
df3 = pd.read_csv(output_file)

# Drop the 'class' column if it exists
if 'class' in df3.columns:
    df3.drop(columns=['class'], inplace=True)

# Save the updated DataFrame to a new CSV file
output_file = 'final_predictions_no_class_new12.csv'
df3.to_csv(output_file, index=False)

print(f"'class' column dropped. Updated file saved as '{output_file}'")
# Matthews Correlation Coefficient (OOF): 0.9851386851107209


xgboost  cv=10
best_params = {
   
    'enable_categorical': True,
    'tree_method': 'hist',
    'device': 'cuda',
    'n_estimators': 360,         
    'learning_rate': 0.1,           
    'max_depth': 17,                
    'colsample_bytree': 0.4,         
    'min_child_weight': 2,           
    'reg_lambda': 67,                
    'subsample': 0.98,              
    'num_parallel_tree': 4,
}

Matthews Correlation Coefficient (OOF): 0.9851386851107209  , cv=10

Submission and Description

Public Score

Select

final_predictions_no_class_new12 (14).csv
Complete · 28s ago
0.98508

In [ ]:
# Step 4: Create Meta-Model Input Data


oof_df = pd.DataFrame({
     'lgbm_oof': trainset_oof['lgbm_oof'],
    'xgb_oof': trainset_oof['xgb_oof'],
    'catboost_oof': oof_preds_catboost,
    'target': y
})


# Optionally, inspect the created DataFrame
print(oof_df.head())

# Create input features (X_meta) and target (y_meta) for the meta-model
X_meta = oof_df[['lgbm_oof', 'xgb_oof','catboost_oof']]
y_meta = oof_df['target']


In [ ]:
# Save the DataFrame as a CSV file
oof_df.to_csv('10_cv_lgbm_xgboost_catboost_oof_train_set_csv.csv', index=False)
x=pd.read_csv('10_cv_lgbm_xgboost_catboost_oof_train_set_csv.csv')
x.head()


In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.linear_model import ElasticNet
import optuna

# Step 4: Hyperparameter Tuning for Meta-Model (ElasticNet) with Optuna
def objective(trial):
    alpha = trial.suggest_float('alpha', 1e-6, 1.0, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0)
    max_iter = trial.suggest_int('max_iter', 100, 2000)

    meta_model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=max_iter, random_state=9)

    skf_meta = StratifiedKFold(n_splits=4, shuffle=True, random_state=9)
    meta_pred_oof = cross_val_predict(meta_model, X_meta, y_meta, cv=skf_meta, method='predict')
    meta_pred_binary = (meta_pred_oof > 0.5).astype(int)

    mcc = matthews_corrcoef(y_meta, meta_pred_binary)

    return mcc

# Run Optuna optimization
# Create a TPE sampler with a fixed seed
#tpe_sampler = optuna.samplers.TPESampler(seed=42)

# Create the study with the specified sampler
#study = optuna.create_study(direction='maximize', sampler=tpe_sampler)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Retrieve the best hyperparameters
best_params = study.best_params
print(f"Best hyperparameters for the meta model: {best_params}")

# Step 5: Train Meta-Model with the Best Hyperparameters
best_meta_model1 = ElasticNet(**best_params, random_state=9)
best_meta_model1.fit(X_meta, y_meta)

In [ ]:
## working on
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.linear_model import LogisticRegression
import optuna
from sklearn.metrics import matthews_corrcoef

# Step 4: Hyperparameter Tuning for Meta-Model (Logistic Regression) with Optuna
def objective(trial):
    C = trial.suggest_float('C', 1e-6, 10.0, log=True)  # Inverse of regularization strength
    max_iter = trial.suggest_int('max_iter', 100, 1000)  # Maximum iterations
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga', 'lbfgs'])  # Solvers

    # Create the Logistic Regression model with sampled parameters
    meta_model = LogisticRegression(C=C, max_iter=max_iter, solver=solver, random_state=9)

    # Stratified K-Fold cross-validation
    skf_meta = StratifiedKFold(n_splits=4, shuffle=True, random_state=9)
    meta_pred_oof = cross_val_predict(meta_model, X_meta, y_meta, cv=skf_meta, method='predict')

    # Calculate MCC
    mcc = matthews_corrcoef(y_meta, meta_pred_oof)
    return mcc

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Retrieve the best hyperparameters
best_params = study.best_params
print(f"Best hyperparameters for the meta model: {best_params}")

# Step 5: Train Meta-Model with the Best Hyperparameters
best_meta_model2 = LogisticRegression(**best_params, random_state=9)
best_meta_model2.fit(X_meta, y_meta)


In [ ]:
# Step 7: Generate Predictions on Test Set
#xgb1_test_oof = best_model1.predict_proba(df_test)[:, 1]
#lgbm_test_oof = best_model2.predict_proba(df_test)[:, 1]
oof_test_preds_catboost = model4.predict_proba(df_test)[:, 1]

test_meta_df = pd.DataFrame({
     'lgbm_oof': testset_oof['lgbm_oof'],
    'xgb_oof': testset_oof['xgb_oof'],
    'catboost_oof': oof_test_preds_catboost
})

# Standardize the test meta data
#test_meta_scaled = scaler.transform(test_meta_df)
#test_predictions_prob = elastic_net_model.predict_proba(test_meta_df)[:, 1]

test_predictions_prob = best_meta_model2.predict(test_meta_df)

# Convert probabilities to binary class labels
test_pred_binary = (test_predictions_prob > 0.5).astype(int)

df_sample_submission['predictions'] = test_pred_binary
df_sample_submission['predictions'] = df_sample_submission['predictions'].replace({0: 'e', 1: 'p'})
df_sample_submission.head()
# Save the final DataFrame to CSV
output_file = 'final_predictions_xgboost_stacked_model.csv'
df_sample_submission.to_csv(output_file, index=False)

# Load the CSV file
df3 = pd.read_csv('final_predictions_xgboost_stacked_model.csv')

# Drop the 'class' column
df3.drop(columns=['class'], inplace=True)

# Save the updated DataFrame to a new CSV file
output_file = 'final_predictions_no_class_new_xg_boost_stacked1234.csv'
df3.to_csv(output_file, index=False)

print(f"'class' column dropped. Updated file saved as '{output_file}'")


In [ ]:
# Save the DataFrame as a CSV file
test_meta_df.to_csv('10_cv_lgbm_xgboost_catboost_oof_test_set_csv.csv', index=False)

In [ ]:
x=pd.read_csv('10_cv_lgbm_xgboost_catboost_oof_test_set_csv.csv')
x.head()

In [ ]:
test_meta_df = pd.DataFrame({
     'lgbm_oof': testset_oof['lgbm_oof'],
    'xgb_oof': testset_oof['xgb_oof'],
    'catboost_oof': oof_preds_catboost
})

test_predictions_prob = best_meta_model2.predict(test_meta_df)

# Convert probabilities to binary class labels
test_pred_binary = (test_predictions_prob > 0.5).astype(int)

df_sample_submission['predictions'] = test_pred_binary
df_sample_submission['predictions'] = df_sample_submission['predictions'].replace({0: 'e', 1: 'p'})
df_sample_submission.head()
# Save the final DataFrame to CSV
output_file = 'final_predictions_xgboost_stacked_model.csv'
df_sample_submission.to_csv(output_file, index=False)

# Load the CSV file
df3 = pd.read_csv('final_predictions_xgboost_stacked_model.csv')

# Drop the 'class' column
df3.drop(columns=['class'], inplace=True)

# Save the updated DataFrame to a new CSV file
output_file = 'final_predictions_no_class_new_xg_boost_stacked1234.csv'
df3.to_csv(output_file, index=False)

print(f"'class' column dropped. Updated file saved as '{output_file}'")

In [ ]:
### Performing the hyperparameter tuning with on lgbm which score lb score as 98497

#LGBM OPTUNA TUNING MUST TRY because it will give 98497 lb score 

def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 400, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 2500),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 100.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 100.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 15)
    }
    
    model = LGBMClassifier(**params, random_state=0, objective='binary', verbosity = -1)
    X, X_val, y, y_val = train_test_split(X_train, y_train, random_state=0, stratify=y_train, test_size=0.2, shuffle=True)
    model.fit(X, y)
    y_pred = model.predict(X_val)
    return matthews_corrcoef(y_val, y_pred)
    

sqlite_db = "sqlite:///lgbm.db"
study_name = "lgbm"
optimize = False
if optimize:
    study = optuna.create_study(storage=sqlite_db, study_name=study_name, 
                                sampler=TPESampler(n_startup_trials=50, multivariate=True, seed=0),
                                direction="maximize", load_if_exists=True)

    study.optimize(objective, n_trials=150)
    print(f"best optimized MCC: {study.best_value:0.5f}") # 0.98521
    print(f"best hyperparameters: {study.best_params}") 
    lgbm_params = study.best_params

In [ ]:
import optuna
from lightgbm import LGBMClassifier
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef
import pandas as pd

# Define the objective function for Optuna
def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 400, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 2500),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 100.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 100.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'device': 'GPU'
    }
    
    model = LGBMClassifier(**params, random_state=0, objective='binary', verbosity=-1)
    X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0, stratify=y, test_size=0.2, shuffle=True)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return matthews_corrcoef(y_val, y_pred)

# Initialize Optuna study and optimize
study = optuna.create_study(sampler=TPESampler(), direction='maximize')
study.optimize(objective, n_trials=100)  # Adjust n_trials as needed

# Retrieve best hyperparameters
best_params = study.best_params

# Train the model on the full training set with the best hyperparameters
best_model = LGBMClassifier(**best_params, random_state=0, objective='binary', verbosity=-1)
best_model.fit(X, y)

# Predict on test data
test_pred_prob = best_model.predict_proba(df_test)[:, 1]

# Convert probabilities to binary class labels
test_pred_binary = (test_pred_prob > 0.5).astype(int)

# Prepare submission
df_sample_submission['predictions'] = test_pred_binary
df_sample_submission['predictions'] = df_sample_submission['predictions'].replace({0: 'e', 1: 'p'})
df_sample_submission.head()

# Save the final DataFrame to CSV
output_file = 'final_predictions_lgbm.csv'
df_sample_submission.to_csv(output_file, index=False)

# Load the CSV file to modify
df3 = pd.read_csv(output_file)

# Drop the 'class' column if it exists
if 'class' in df3.columns:
    df3.drop(columns=['class'], inplace=True)

# Save the updated DataFrame to a new CSV file
output_file = 'final_predictions_no_class_new12.csv'
df3.to_csv(output_file, index=False)

print(f"'class' column dropped. Updated file saved as '{output_file}'")


[I 2024-08-26 14:19:49,014] Trial 0 finished with value: 0.9848939633638614 and parameters: {'num_leaves': 730, 'learning_rate': 0.034046059791858485, 'n_estimators': 1513, 'subsample_for_bin': 176028, 'min_child_samples': 494, 'reg_alpha': 2.0417397933460876e-05, 'reg_lambda': 4.454406348639976e-06, 'colsample_bytree': 0.48663514847243605, 'subsample': 0.6018624080090369, 'max_depth': 15}. Best is trial 0 with value: 0.9848939633638614.


[I 2024-08-26 23:09:13,798] Trial 33 finished with value: 0.9851800323425373 and parameters: {'num_leaves': 705, 'learning_rate': 0.014413440850606295, 'n_estimators': 2496, 'subsample_for_bin': 203526, 'min_child_samples': 285, 'reg_alpha': 2.7046833821279917e-05, 'reg_lambda': 7.38502567343188e-07, 'colsample_bytree': 0.3675170902152378, 'subsample': 0.6621847806842337, 'max_depth': 13}. Best is trial 33 with value: 0.9851800323425373.

### Working on 2 models for stacking

In [ ]:
# Load your train and test OOF predictions
testset_oof = pd.read_csv("/kaggle/working/10_cv_lgbm_xgboost_oof_test_set_csv.csv")
trainset_oof = pd.read_csv("/kaggle/working/10_cv_lgbm_xgboost_oof_train_set_csv.csv")

In [ ]:


# Create input features (X_meta) and target (y_meta) for the meta-model
X_meta = trainset_oof[['lgbm_oof', 'xgb_oof']]
y_meta = trainset_oof['target']


In [ ]:
## working on
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.linear_model import LogisticRegression
import optuna
from sklearn.metrics import matthews_corrcoef

# Step 4: Hyperparameter Tuning for Meta-Model (Logistic Regression) with Optuna
def objective(trial):
    C = trial.suggest_float('C', 1e-6, 10.0, log=True)  # Inverse of regularization strength
    max_iter = trial.suggest_int('max_iter', 100, 1000)  # Maximum iterations
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga', 'lbfgs'])  # Solvers

    # Create the Logistic Regression model with sampled parameters
    meta_model = LogisticRegression(C=C, max_iter=max_iter, solver=solver, random_state=9)

    # Stratified K-Fold cross-validation
    skf_meta = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
    meta_pred_oof = cross_val_predict(meta_model, X_meta, y_meta, cv=skf_meta, method='predict')

    # Calculate MCC
    mcc = matthews_corrcoef(y_meta, meta_pred_oof)
    return mcc

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Retrieve the best hyperparameters
best_params = study.best_params
print(f"Best hyperparameters for the meta model: {best_params}")

# Step 5: Train Meta-Model with the Best Hyperparameters
best_meta_model2 = LogisticRegression(**best_params, random_state=9)
best_meta_model2.fit(X_meta, y_meta)


In [ ]:
test_meta1=testset_oof[['lgbm_oof', 'xgb_oof']]
test_predictions_prob = best_meta_model2.predict(test_meta1)

# Convert probabilities to binary class labels
test_pred_binary = (test_predictions_prob > 0.5).astype(int)

df_sample_submission['predictions'] = test_pred_binary
df_sample_submission['predictions'] = df_sample_submission['predictions'].replace({0: 'e', 1: 'p'})
df_sample_submission.head()
# Save the final DataFrame to CSV
output_file = 'final_predictions_xgboost_stacked_model.csv'
df_sample_submission.to_csv(output_file, index=False)

# Load the CSV file
df3 = pd.read_csv('final_predictions_xgboost_stacked_model.csv')

# Drop the 'class' column
df3.drop(columns=['class'], inplace=True)

# Save the updated DataFrame to a new CSV file
output_file = '_optuna_final_predictions_no_class_new_xg_boost_stacked12.csv'
df3.to_csv(output_file, index=False)

print(f"'class' column dropped. Updated file saved as '{output_file}'")

In [ ]:
### catboost with best params 
import lightgbm as lgb
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import matthews_corrcoef

# Best parameters for LightGBM
best_params = {
    'task_type': 'GPU','n_estimators': 5939, 'learning_rate': 0.011746226160973147, 'random_strength': 0.6406712628822828, 'max_bin': 71, 'depth': 11, 'l2_leaf_reg': 6.236947205167646, 'grow_policy': 'SymmetricTree', 'bootstrap_type': 'Bernoulli', 'subsample': 0.4963564081845081
}

# Initialize LightGBM with the best parameters
best_model9 = CatBoostClassifier(**best_params)

# Train the model on the entire training data
best_model9.fit(X, y)

# Print feature importances
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_model9.feature_importances_
}).sort_values(by='Importance', ascending=False)
print("\nFeature importances for LightGBM:")
print(importance_df.head(20))

# Predict on test data
test_pred_prob = best_model9.predict_proba(df_test)[:, 1]

# Convert probabilities to binary class labels
test_pred_binary = (test_pred_prob > 0.5).astype(int)

# Prepare submission
df_sample_submission['predictions'] = test_pred_binary
df_sample_submission['predictions'] = df_sample_submission['predictions'].replace({0: 'e', 1: 'p'})
df_sample_submission.head()

# Save the final DataFrame to CSV
output_file = 'final_predictions_lgbm.csv'
df_sample_submission.to_csv(output_file, index=False)

# Load the CSV file to modify
df3 = pd.read_csv(output_file)

# Drop the 'class' column if it exists
if 'class' in df3.columns:
    df3.drop(columns=['class'], inplace=True)

# Save the updated DataFrame to a new CSV file
output_file = 'final_predictions_no_class_new12_lgbm.csv'
df3.to_csv(output_file, index=False)

print(f"'class' column dropped. Updated file saved as '{output_file}'")


In [ ]:
# Return the trained model and test predictions
from catboost import CatBoostClassifier
catboost_model = CatBoostClassifier()
model4=catboost_model.load_model("/kaggle/input/catboost-oofs-and-best-model/catboost_final_model.cbm")
print(model4)
# Print the model parameters
print("Model Parameters:")
print(model4.get_params())

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import matthews_corrcoef
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier

# List of models to evaluate
models = [
    ("Logistic Regression", LogisticRegression(random_state=9)),
    ("Ridge Classifier", RidgeClassifier(random_state=9)),
    ("SGD Classifier", SGDClassifier(random_state=9)),
    ("Support Vector Classifier (SVC)", SVC(random_state=9)),
    ("Linear SVC", LinearSVC(random_state=9)),
    ("K-Nearest Neighbors", KNeighborsClassifier()),
    ("Decision Tree", DecisionTreeClassifier(random_state=9)),
    ("Random Forest", RandomForestClassifier(random_state=9)),
    ("Gradient Boosting", GradientBoostingClassifier(random_state=9)),
    ("AdaBoost", AdaBoostClassifier(random_state=9)),
    ("Extra Trees", ExtraTreesClassifier(random_state=9)),
    ("XGBoost", XGBClassifier(use_label_encoder=False, random_state=9, eval_metric='logloss')),
    ("Naive Bayes", GaussianNB())
]

# Initialize Stratified K-Fold cross-validation
skf_meta = StratifiedKFold(n_splits=4, shuffle=True, random_state=9)

# Iterate through each model
for name, model7 in models:
    # Generate out-of-fold predictions
    meta_pred_oof = cross_val_predict(model7, X_meta, y_meta, cv=skf_meta, method='predict')

    # Calculate MCC for the model
    mcc = matthews_corrcoef(y_meta, meta_pred_oof)

    # Print the model name and MCC score
    print(f"{name}: Matthews Correlation Coefficient (MCC) = {mcc:.5f}")

    # Train the model on the entire dataset
    model7.fit(X_meta, y_meta)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import KFold
from catboost import CatBoostClassifier
import gc

# Initialize OOF predictions array
oof_preds_catboost = np.zeros(len(X))  # assuming X is your dataset

# List of saved model paths
model_paths = [
    "/kaggle/input/catboost-oofs-and-best-model/catboost_model_fold_0.cbm",
    "/kaggle/input/catboost-oofs-and-best-model/catboost_model_fold_1.cbm",
    "/kaggle/input/catboost-oofs-and-best-model/catboost_model_fold_2.cbm",
    "/kaggle/input/catboost-oofs-and-best-model/catboost_model_fold_3.cbm",
    "/kaggle/input/catboost-oofs-and-best-model/catboost_model_fold_4.cbm",
    "/kaggle/input/catboost-oofs-and-best-model/catboost_model_fold_5.cbm",
    "/kaggle/input/catboost-oofs-and-best-model/catboost_model_fold_6.cbm",
    "/kaggle/input/catboost-oofs-and-best-model/catboost_model_fold_7.cbm",
    "/kaggle/input/catboost-oofs-and-best-model/catboost_model_fold_8.cbm",
    "/kaggle/input/catboost-oofs-and-best-model/catboost_model_fold_9.cbm"
]

# KFold setup
cv = KFold(n_splits=10, shuffle=True, random_state=9)

for fold, (train_ind,valid_ind) in enumerate(cv.split(X, y)):
    # Load the saved model for the current fold
    model = CatBoostClassifier()
    model.load_model(model_paths[fold])
    X_val = X.iloc[valid_ind]
    # Generate OOF predictions for the validation set
    oof_preds_catboost[valid_ind] = model.predict_proba(X_val)[:, 1]
    # Clean up
    gc.collect()

# The array `oof_preds_catboost` now contains the OOF predictions for all validation sets


In [ ]:
print(X_meta)